# Advent of Code

## 2024-012-013
## 2024 013

https://adventofcode.com/2024/day/13

In [2]:
from sympy import symbols, Eq, solve
import re

# Define variables for calculations
a, b = symbols('a b', integer=True)

# Function to parse machine data from multi-line input
def parse_multiline_machine(lines):
    if len(lines) == 3:
        button_a = re.match(r"Button A: X\+(\d+), Y\+(\d+)", lines[0])
        button_b = re.match(r"Button B: X\+(\d+), Y\+(\d+)", lines[1])
        prize = re.match(r"Prize: X=(\d+), Y=(\d+)", lines[2])
        if button_a and button_b and prize:
            return tuple(map(int, button_a.groups() + button_b.groups() + prize.groups()))
    return None

# Function to compute solutions for a list of machines
def compute_solutions(machines, offset_x=0, offset_y=0):
    results = []
    for Ax, Ay, Bx, By, Px, Py in machines:
        # Apply offset if provided
        Px += offset_x
        Py += offset_y
        
        # Solve the system of equations
        eq1 = Eq(a * Ax + b * Bx, Px)
        eq2 = Eq(a * Ay + b * By, Py)
        solutions = solve((eq1, eq2), (a, b), dict=True)
        valid_solutions = [
            (sol[a], sol[b]) for sol in solutions if sol[a] >= 0 and sol[b] >= 0
        ]
        
        # Find the minimum token cost
        min_cost = float('inf')
        best_sol = None
        for sol_a, sol_b in valid_solutions:
            cost = 3 * sol_a + sol_b
            if cost < min_cost:
                min_cost = cost
                best_sol = (sol_a, sol_b)
        if best_sol:
            results.append((min_cost, best_sol))

    # Calculate total cost and machines solved
    results.sort()
    total_cost = sum(cost for cost, _ in results)
    machines_solved = len(results)

    return total_cost, machines_solved

# Main program
if __name__ == "__main__":
    input_file_path = "input.txt"  # Adjust this to the actual file path

    # Parse input data
    with open(input_file_path, 'r') as f:
        raw_lines = [line.strip() for line in f if line.strip()]

    # Group every 3 lines into a machine
    machines = [parse_multiline_machine(raw_lines[i:i+3]) for i in range(0, len(raw_lines), 3)]
    parsed_machines = [machine for machine in machines if machine is not None]

    # Part 1: Solve without offsets
    part1_total_cost, part1_machines_solved = compute_solutions(parsed_machines)

    # Part 2: Solve with large offsets
    OFFSET = 10_000_000_000_000
    part2_total_cost, part2_machines_solved = compute_solutions(parsed_machines, offset_x=OFFSET, offset_y=OFFSET)

    # Print results
    print("Part 1:")
    print(f"Total tokens spent: {part1_total_cost}")
    print(f"Machines solved: {part1_machines_solved}")

    print("\nPart 2:")
    print(f"Total tokens spent: {part2_total_cost}")
    print(f"Machines solved: {part2_machines_solved}")

Part 1:
Total tokens spent: 37901
Machines solved: 183

Part 2:
Total tokens spent: 77407675412647
Machines solved: 137


In [4]:
import re
import numpy as np
from scipy.optimize import linprog

def parse_multiline_machine(lines):
    """Parses machine data from three lines of input."""
    if len(lines) == 3:
        button_a = re.match(r"Button A: X\+(\d+), Y\+(\d+)", lines[0])
        button_b = re.match(r"Button B: X\+(\d+), Y\+(\d+)", lines[1])
        prize = re.match(r"Prize: X=(\d+), Y=(\d+)", lines[2])
        if button_a and button_b and prize:
            return tuple(map(int, button_a.groups() + button_b.groups() + prize.groups()))
    return None

def compute_solutions(machines, offset_x=0, offset_y=0):
    """Computes the minimum cost solutions for a list of machines."""
    results = []
    for Ax, Ay, Bx, By, Px, Py in machines:
        # Apply offset if provided
        Px += offset_x
        Py += offset_y

        # Coefficients for the linear system
        A = np.array([[Ax, Bx], [Ay, By]])
        b = np.array([Px, Py])

        # Bounds for variables a and b (must be non-negative)
        bounds = [(0, None), (0, None)]

        # Cost function: Minimize 3a + b
        c = np.array([3, 1])

        # Solve the linear system using linprog
        res = linprog(c, A_eq=A, b_eq=b, bounds=bounds, method="highs")

        if res.success and all(x >= 0 for x in res.x):
            a, b = map(int, res.x.round())
            cost = 3 * a + b
            results.append((cost, (a, b)))

    # Sort results by cost
    results.sort()

    # Calculate total cost and machines solved
    total_cost = sum(cost for cost, _ in results)
    machines_solved = len(results)

    return total_cost, machines_solved

if __name__ == "__main__":
    input_file_path = "input.txt"  # Adjust this to the actual file path

    # Parse input data
    with open(input_file_path, 'r') as f:
        raw_lines = [line.strip() for line in f if line.strip()]

    # Group every 3 lines into a machine
    machines = [parse_multiline_machine(raw_lines[i:i+3]) for i in range(0, len(raw_lines), 3)]
    parsed_machines = [machine for machine in machines if machine is not None]

    # Part 1: Solve without offsets
    part1_total_cost, part1_machines_solved = compute_solutions(parsed_machines)

    # Part 2: Solve with large offsets
    OFFSET = 10_000_000_000_000
    part2_total_cost, part2_machines_solved = compute_solutions(parsed_machines, offset_x=OFFSET, offset_y=OFFSET)

    # Print results
    print("Part 1:")
    print(f"Total tokens spent: {part1_total_cost}")
    print(f"Machines solved: {part1_machines_solved}")

    print("\nPart 2:")
    print(f"Total tokens spent: {part2_total_cost}")
    print(f"Machines solved: {part2_machines_solved}")

Part 1:
Total tokens spent: 95311
Machines solved: 266

Part 2:
Total tokens spent: 151058428761624
Machines solved: 313


In [7]:
import re
from scipy.optimize import linprog

def parse_multiline_machine(lines):
    """Parses machine data from three lines of input."""
    if len(lines) == 3:
        button_a = re.match(r"Button A: X\+(\d+), Y\+(\d+)", lines[0])
        button_b = re.match(r"Button B: X\+(\d+), Y\+(\d+)", lines[1])
        prize = re.match(r"Prize: X=(\d+), Y=(\d+)", lines[2])
        if button_a and button_b and prize:
            return tuple(map(int, button_a.groups() + button_b.groups() + prize.groups()))
    return None

def compute_solutions(machines, offset_x=0, offset_y=0):
    """Computes the minimum cost solutions for a list of machines."""
    results = []
    for Ax, Ay, Bx, By, Px, Py in machines:
        # Apply offset if provided
        Px += offset_x
        Py += offset_y

        # Coefficients for the linear system
        A = [[Ax, Bx], [Ay, By]]
        b = [Px, Py]

        # Cost function: Minimize 3a + b
        c = [3, 1]

        # Solve the linear system using linprog
        res = linprog(c, A_eq=A, b_eq=b, bounds=[(0, None), (0, None)], method="highs")

        if res.success and all(x >= 0 for x in res.x):
            a, b = map(round, res.x)
            cost = 3 * a + b
            results.append((cost, (a, b)))

    # Sort results by cost
    results.sort()

    # Calculate total cost and machines solved
    total_cost = sum(cost for cost, _ in results)
    machines_solved = len(results)

    return total_cost, machines_solved

if __name__ == "__main__":
    input_file_path = "input.txt"  # Adjust this to the actual file path

    # Parse input data
    with open(input_file_path, 'r') as f:
        raw_lines = [line.strip() for line in f if line.strip()]

    # Group every 3 lines into a machine
    machines = [parse_multiline_machine(raw_lines[i:i+3]) for i in range(0, len(raw_lines), 3)]
    parsed_machines = [machine for machine in machines if machine is not None]

    # Part 1: Solve without offsets
    part1_total_cost, part1_machines_solved = compute_solutions(parsed_machines)

    # Part 2: Solve with large offsets
    OFFSET = 10_000_000_000_000
    part2_total_cost, part2_machines_solved = compute_solutions(parsed_machines, offset_x=OFFSET, offset_y=OFFSET)

    # Print results
    print("Part 1:")
    print(f"Total tokens spent: {part1_total_cost}")
    print(f"Machines solved: {part1_machines_solved}")

    print("\nPart 2:")
    print(f"Total tokens spent: {part2_total_cost}")
    print(f"Machines solved: {part2_machines_solved}")

Part 1:
Total tokens spent: 95311
Machines solved: 266

Part 2:
Total tokens spent: 151058428761624
Machines solved: 313


In [8]:
import re
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Function to parse machine data from multi-line input
def parse_multiline_machine(lines):
    if len(lines) == 3:
        button_a = re.match(r"Button A: X\+(\d+), Y\+(\d+)", lines[0])
        button_b = re.match(r"Button B: X\+(\d+), Y\+(\d+)", lines[1])
        prize = re.match(r"Prize: X=(\d+), Y=(\d+)", lines[2])
        if button_a and button_b and prize:
            return tuple(map(int, button_a.groups() + button_b.groups() + prize.groups()))
    return None

# Function to compute solutions for a list of machines
def compute_solutions(machines, offset_x=0, offset_y=0):
    results = []
    for Ax, Ay, Bx, By, Px, Py in machines:
        # Apply offset if provided
        Px += offset_x
        Py += offset_y

        # Define the problem
        prob = LpProblem("Minimize_Tokens", LpMinimize)

        # Define variables (a and b must be integers)
        a = LpVariable("a", lowBound=0, cat="Integer")
        b = LpVariable("b", lowBound=0, cat="Integer")

        # Objective function: Minimize 3a + b
        prob += 3 * a + b

        # Constraints
        prob += a * Ax + b * Bx == Px, "X_constraint"
        prob += a * Ay + b * By == Py, "Y_constraint"

        # Solve the problem
        prob.solve()

        # Check if the solution is valid
        if a.varValue is not None and b.varValue is not None:
            a_val = int(a.varValue)
            b_val = int(b.varValue)
            cost = 3 * a_val + b_val
            results.append((cost, (a_val, b_val)))

    # Sort results by cost
    results.sort()

    # Calculate total cost and machines solved
    total_cost = sum(cost for cost, _ in results)
    machines_solved = len(results)

    return total_cost, machines_solved

# Main program
if __name__ == "__main__":
    input_file_path = "input.txt"  # Adjust this to the actual file path

    # Parse input data
    with open(input_file_path, 'r') as f:
        raw_lines = [line.strip() for line in f if line.strip()]

    # Group every 3 lines into a machine
    machines = [parse_multiline_machine(raw_lines[i:i+3]) for i in range(0, len(raw_lines), 3)]
    parsed_machines = [machine for machine in machines if machine is not None]

    # Part 1: Solve without offsets
    part1_total_cost, part1_machines_solved = compute_solutions(parsed_machines)

    # Part 2: Solve with large offsets
    OFFSET = 10_000_000_000_000
    part2_total_cost, part2_machines_solved = compute_solutions(parsed_machines, offset_x=OFFSET, offset_y=OFFSET)

    # Print results
    print("Part 1:")
    print(f"Total tokens spent: {part1_total_cost}")
    print(f"Machines solved: {part1_machines_solved}")

    print("\nPart 2:")
    print(f"Total tokens spent: {part2_total_cost}")
    print(f"Machines solved: {part2_machines_solved}")

Part 1:
Total tokens spent: 122672
Machines solved: 320

Part 2:
Total tokens spent: 153334949210422
Machines solved: 320
